### W.O.E. Rank


Created By: mac-0

#### Step 0:
Importing Libraries

In [1]:
import sqlite3
import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np

#### Step 1:

Calculate Total League win-rate by Race to give us an expected win-rate by race which will be used to calculated the "expected" part of this ranking.

This is currently calculated using only the same league data that the "WOE" rank will. This means that if we are creating one for a season (e.g. Season 22 CCL, the expected win-rate will be based only off of games played in Season 22). I will consider revisiting this to allow for larger sample size.

In [2]:
conn = sqlite3.connect('ccl_current.sqlite')
win_rates_by_race_query = '''
select
    t.race,
    sum(tm.win) + sum(tm.draw) + sum(tm.loss) as games,
    sum(tm.win) as wins,
    sum(tm.draw) as draws,
    sum(tm.loss) as losses
    --(sum(tm.win+0.0) + sum(tm.draw+0.0)/2)/(sum(tm.win+0.0) + sum(tm.draw+0.0) + sum(tm.loss+0.0)) as win_percent
from teammatches tm
inner join teams t
    on tm.idteamlisting = t.id
group by 
    t.race
    '''
win_rates_by_race = pd.read_sql_query(win_rates_by_race_query,conn)
win_rates_by_race.head()

,race,games,wins,draws,losses
0,Amazon,125,67,25,33
1,Bretonnia,103,46,24,33
2,Chaos,370,131,74,165
3,ChaosDwarf,278,145,50,83
4,DarkElf,276,122,45,109


#### Step 2:

Calculate Total League win-rate by Race for all Coaches

In [3]:
win_rates_by_race_and_coach_query = '''
select
    tm.coach,
    t.race,
    sum(tm.win) + sum(tm.draw) + sum(tm.loss) as games,
    sum(tm.win) as wins,
    sum(tm.draw) as draws,
    sum(tm.loss) as losses
    --(sum(tm.win+0.0) + sum(tm.draw+0.0)/2)/(sum(tm.win+0.0) + sum(tm.draw+0.0) + sum(tm.loss+0.0)) as win_percent
from teammatches tm
inner join teams t
    on tm.idteamlisting = t.id
group by 
    tm.coach,
    t.race
    '''

win_rates_by_race_and_coach = pd.read_sql_query(win_rates_by_race_and_coach_query,conn)
win_rates_by_race_and_coach.head()

,coach,race,games,wins,draws,losses
0,+Kor+,Chaos,4,2,0,2
1,--CD--,Norse,1,0,0,1
2,0beron,Orc,1,0,0,1
3,0rbit,Amazon,2,2,0,0
4,0rbit,Ogre,1,1,0,0


In [4]:
expected_win_rates = pd.merge(win_rates_by_race_and_coach, win_rates_by_race, how='left',
                             left_on='race',right_on='race')
expected_win_rates.head()

,coach,race,games_x,wins_x,draws_x,losses_x,games_y,wins_y,draws_y,losses_y
0,+Kor+,Chaos,4,2,0,2,370,131,74,165
1,--CD--,Norse,1,0,0,1,147,72,22,53
2,0beron,Orc,1,0,0,1,416,198,76,142
3,0rbit,Amazon,2,2,0,0,125,67,25,33
4,0rbit,Ogre,1,1,0,0,108,37,27,44


In [5]:
expected_win_rates['expected_wins'] = \
    expected_win_rates['games_x'] * ((expected_win_rates['wins_y'] - expected_win_rates['wins_x']) + (expected_win_rates['draws_y'] - expected_win_rates['draws_x'])/2)/(expected_win_rates['games_y'] - expected_win_rates['games_x'])

expected_win_rates['wins_over_expected'] = (expected_win_rates['wins_x'] + expected_win_rates['draws_x']/2) - expected_win_rates['expected_wins']
expected_win_rates.sort_values(by='wins_over_expected',ascending=False).head()

,coach,race,games_x,wins_x,draws_x,losses_x,games_y,wins_y,draws_y,losses_y,expected_wins,wins_over_expected
154,Crucifer,ChaosDwarf,23,17,5,1,278,145,50,83,13.574510,5.925490
436,JimmyFantastic,ChaosDwarf,24,17,5,2,278,145,50,83,14.220472,5.279528
729,Razzle Storm,Kislev,10,7,2,1,58,23,9,26,4.062500,3.937500
193,Dick_Delaware,ChaosDwarf,22,14,6,2,278,145,50,83,13.148438,3.851562
1036,dionysian,ChaosDwarf,33,23,1,9,278,145,50,83,19.732653,3.767347


In [6]:
expected_win_rates.groupby('coach').sum().sort_values(by='wins_over_expected',ascending=False)

,games_x,wins_x,draws_x,losses_x,games_y,wins_y,draws_y,losses_y,expected_wins,wins_over_expected
coach,,,,,,,,,,
Crucifer,23,17,5,1,278,145,50,83,13.574510,5.925490
JimmyFantastic,25,17,6,2,643,295,132,216,14.743824,5.256176
Missuh,14,10,2,2,646,253,119,274,6.531863,4.468137
Dick_Delaware,26,17,6,3,511,273,79,159,15.585119,4.414881
dionysian,34,24,1,9,539,262,103,174,20.280730,4.219270
Metanol,14,11,1,2,416,198,76,142,7.818408,3.681592
SvenRutabega,26,13,8,5,365,150,82,133,13.345133,3.654867
AndyDavo,9,7,1,1,370,131,74,165,4.001385,3.498615
Willking15,22,13,6,3,638,306,119,213,12.509852,3.490148
